In [1]:
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
import cv2 as cv
import numpy as np
from keras.models import load_model

In [2]:
# Load the pre-trained model
model = load_model('best_model.keras') 

In [3]:

# Dictionary mapping numeric labels to alphabet letters
dict_word = {0:'A', 1:'B', 2:'C', 3:'D', 4:'E', 5:'F', 6:'G', 7:'H', 8:'I', 9:'J',
             10:'K', 11:'L', 12:'M', 13:'N', 14:'O', 15:'P', 16:'Q', 17:'R', 18:'S',
             19:'T', 20:'U', 21:'V', 22:'W', 23:'X', 24:'Y', 25:'Z'}

In [4]:

# Function to preprocess and predict image
def predict_image(img_path):
    try:
        # Load and preprocess the image
        img = cv.imread(img_path)
        img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)  # Convert to grayscale
        img = cv.medianBlur(img, 5)  # Apply median blur
        ret, img = cv.threshold(img, 75, 180, cv.THRESH_BINARY)  # Apply thresholding
        element = cv.getStructuringElement(cv.MORPH_RECT, (90, 90))
        img = cv.morphologyEx(img, cv.MORPH_GRADIENT, element)  # Apply morphological gradient
        img = img / 255.0  # Normalize image
        img = cv.resize(img, (28, 28))  # Resize to 28x28 pixels
        img = np.reshape(img, (1, 28, 28, 1))  # Reshape for model input

        # Predict using the model
        pred_label = dict_word[np.argmax(model.predict(img))]  # Predict using the model
        return pred_label

    except Exception as e:
        messagebox.showerror("Error", f"Error predicting image: {e}")
        return None

In [5]:
def select_image():
    file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg;*.jpeg;*.png")])
    if file_path:
        pred_label = predict_image(file_path)
        if pred_label:
            # Display the prediction
            lbl_prediction.config(text=f"Prediction: {pred_label}")

            # Display the image in the panel
            img = Image.open(file_path)
            img = img.resize((400, 400), Image.LANCZOS)  # Use LANCZOS for resizing
            img = ImageTk.PhotoImage(img)
            panel.config(image=img)
            panel.image = img 

In [6]:

# Create the main application window
root = tk.Tk()
root.title("Image Predictor")

# Set window size and position
window_width = 800
window_height = 600
screen_width = root.winfo_screenwidth()
screen_height = root.winfo_screenheight()
x_position = int((screen_width - window_width) / 2)
y_position = int((screen_height - window_height) / 2)
root.geometry(f"{window_width}x{window_height}+{x_position}+{y_position}")

# Change background color
root.configure(bg='lightblue')  # Adjust the color as desired

# Create and pack widgets with larger font and padding
lbl_info = tk.Label(root, text="Select an image to predict:", font=("Helvetica", 20), bg='lightblue')
lbl_info.pack(pady=20)

# Frame for holding the button and text entry
frame = tk.Frame(root, bg='lightblue')
frame.pack(pady=20)

# Create prediction label widget
lbl_prediction = tk.Label(root, text="", font=("Helvetica", 18, "bold"), fg='green', bg='lightblue')
lbl_prediction.pack(pady=10)

# Create a panel to display the image
panel = tk.Label(root, bg='white', width=400, height=400)
panel.pack(pady=20)

# Create select button
btn_select = tk.Button(frame, text="Select Image", font=("Helvetica", 16), command=select_image)
btn_select.pack(pady=10)

In [7]:

# Run the main loop
root.mainloop()



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 333ms/step
